# Metacritic Score Datasets (Feb 2023)

## Load Packages and Functions

In [51]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib


## Load Data 

In [52]:
df_movies = pd.read_csv('../../data/processed/df_movies_processed.csv')

In [53]:
df_movies.columns

Index(['Unnamed: 0', 'metascore', 'rating', 'release_date', 'sort_no',
       'summary', 'title', 'user_score', 'release_year', 'release_month',
       'release_quarter', 'release_yearweek', 'title_len', 'summary_len',
       'title_wordlen', 'summary_wordlen', 'title_avg_wordlength',
       'summary_avg_wordlength', 'title_capcount', 'summary_capcount',
       'title_punccount', 'summary_punccount', 'title_sentcount',
       'summary_sentcount', 'title_uniq', 'summary_uniq',
       'title_uniq_vs_words', 'summary_uniq_vs_words', 'title_stopcount',
       'summary_stopcount', 'title_stopwords_vs_words',
       'summary_stopwords_vs_words', 'title_clean', 'summary_clean',
       'title_clean_num', 'summary_clean_num', 'title_topic', 'summary_topic'],
      dtype='object')

In [54]:
df_movies.drop(['Unnamed: 0','release_date','summary', 'title','title_clean','sort_no',
                'summary_clean', 'title_clean_num', 'summary_clean_num',
                'title_punccount','summary_punccount',
               'title_wordlen', 'summary_wordlen','title_uniq', 'summary_uniq'],axis=1,inplace=True)

In [55]:
df_movies.select_dtypes('object').value_counts()

rating   
R            4938
Not Rated    4025
PG-13        2586
PG           1472
-1           1124
TV-MA         260
G             230
TV-14         149
TV-PG         140
Approved      134
Passed         94
NC-17          83
TV-G           40
GP             23
M               9
M/PG            8
Open            6
TV-Y7           2
MA-17           1
TV-Y7-FV        1
dtype: int64

## Encoding and Train-Test Split

In [56]:
from sklearn.preprocessing import LabelEncoder
def label_encode_columns(dataframe, columns):
    """Label encodes the specified columns of a dataframe"""
    encoded_data = {}
    for column in columns:
        le = LabelEncoder()
        dataframe[column] = le.fit_transform(dataframe[column].astype('str'))
        le_classes = le.classes_
        encoded_data[column] = {"label_encoder": le, "classes": le_classes}
    return {"dataframe": dataframe, "encoded_data": encoded_data}

In [57]:
movies_encoding = label_encode_columns(df_movies, df_movies.select_dtypes('object').columns.values)

In [58]:
df_movies_encoded = movies_encoding['dataframe']

In [63]:
movies_encoding['encoded_data']['rating']['label_encoder'].transform(movies_encoding['encoded_data']['rating']['classes'])
movies_encoding['encoded_data']['rating']['classes']

array(['-1', 'Approved', 'G', 'GP', 'M', 'M/PG', 'MA-17', 'NC-17',
       'Not Rated', 'Open', 'PG', 'PG-13', 'Passed', 'R', 'TV-14', 'TV-G',
       'TV-MA', 'TV-PG', 'TV-Y7', 'TV-Y7-FV'], dtype=object)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_movies.drop('metascore',axis=1), 
                                                    df_movies['metascore'], 
                                                    test_size=0.30, random_state=42)

In [10]:
y_train.value_counts()

64    261
65    256
68    252
67    251
66    250
     ... 
8       4
7       3
99      3
5       2
6       1
Name: metascore, Length: 97, dtype: int64

In [11]:
X_train.columns

Index(['rating', 'user_score', 'release_year', 'release_month',
       'release_quarter', 'release_yearweek', 'title_len', 'summary_len',
       'title_avg_wordlength', 'summary_avg_wordlength', 'title_capcount',
       'summary_capcount', 'title_sentcount', 'summary_sentcount',
       'title_uniq_vs_words', 'summary_uniq_vs_words', 'title_stopcount',
       'summary_stopcount', 'title_stopwords_vs_words',
       'summary_stopwords_vs_words', 'title_topic', 'summary_topic'],
      dtype='object')

In [12]:
X_train.head()

rating  user_score  release_year  release_month  release_quarter  \
1758        8        -1.0        2023.0            2.0              1.0   
12152       7         6.5        2007.0            8.0              3.0   
10598       8         6.6        2014.0           10.0              4.0   
852        13         7.6        2022.0           10.0              4.0   
1494        0         8.0        2005.0            7.0              3.0   

       release_yearweek  title_len  summary_len  title_avg_wordlength  \
1758                7.0         11          497                  11.0   
12152              32.0          7          222                   7.0   
10598              40.0         13          172                   6.5   
852                40.0          9          172                   4.5   
1494               26.0          9          242                   4.5   

       summary_avg_wordlength  ...  title_sentcount  summary_sentcount  \
1758                 6.291139  ...                1                  3   
12152                6.166667  ...                1                  3   
10598                7.166667  ...                1                  1   
852                  5.058824  ...                1                  2   
1494                 6.722222  ...                1                  2   

       title_uniq_vs_words  summary_uniq_vs_words  title_stopcount  \
1758                   1.0               0.797468                0   
12152                  1.0               0.888889                0   
10598                  1.0               0.958333                0   
852                    1.0               0.882353                0   
1494                   1.0               0.944444                0   

       summary_stopcount  title_stopwords_vs_words  \
1758                  28                       0.0   
12152                 14                       0.0   
10598                  7                       0.0   
852                   14                       0.0   
1494                  13                       0.0   

       summary_stopwords_vs_words  title_topic  summary_topic  
1758                     0.354430            2             -1  
12152                    0.388889           -1             -1  
10598                    0.291667            0             -1  
852                      0.411765            0             -1  
1494                     0.361111           -1              5  

[5 rows x 22 columns]

In [13]:
X_train['title_uniq_vs_words'].max()

1.0

## Model

In [14]:
#linear_reg = LinearRegression()
#decision_tree_reg = DecisionTreeRegressor(random_state = 42)
random_forest_reg = RandomForestRegressor(random_state = 42)
#xgboost_reg = XGBRegressor(seed = 42)

# Step 4: Hyperparameter Tuning
linear_reg_params = {}
decision_tree_reg_params = {'max_depth': [3, 5, 7]}
random_forest_reg_params = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}
xgboost_reg_params = {'learning_rate': [0.1, 0.3, 0.5], 'max_depth': [3, 5, 7]}

#linear_reg_grid = GridSearchCV(linear_reg, linear_reg_params, cv=5)
#decision_tree_reg_grid = GridSearchCV(decision_tree_reg, decision_tree_reg_params, cv=5)
random_forest_reg_grid = GridSearchCV(random_forest_reg, random_forest_reg_params, cv=5)
#xgboost_reg_grid = GridSearchCV(xgboost_reg, xgboost_reg_params, cv=5)

#linear_reg_grid.fit(X_train, y_train)
#decision_tree_reg_grid.fit(X_train, y_train)
random_forest_reg_grid.fit(X_train, y_train)
#xgboost_reg_grid.fit(X_train, y_train)

# Step 5: Model Training
#linear_reg_model = linear_reg_grid.best_estimator_
#decision_tree_reg_model = decision_tree_reg_grid.best_estimator_
random_forest_reg_model = random_forest_reg_grid.best_estimator_
#xgboost_reg_model = xgboost_reg_grid.best_estimator_

In [15]:
#linear_reg_model.fit(X_train, y_train)
#print('Fitter LG')
#decision_tree_reg_model.fit(X_train, y_train)
#print('Fitter DT')
random_forest_reg_model.fit(X_train, y_train)
print('Fitter RF')
#xgboost_reg_model.fit(X_train, y_train)
#print('Fitter XGB')

Fitter RF


In [16]:
#y_pred_linear = linear_reg_model.predict(X_test)
#y_pred_dt = decision_tree_reg_model.predict(X_test)
y_pred_rf = random_forest_reg_model.predict(X_test)
#y_pred_xgb = xgboost_reg_model.predict(X_test)

In [17]:
# Step 5: Evaluate the model performance
print("RF: r-score = ",metrics.r2_score(y_test, y_pred_rf), ", MSE = ", 
      metrics.mean_squared_error(y_test, y_pred_rf),", MAPE = ",
      metrics.mean_absolute_percentage_error(y_test, y_pred_rf))

#LinearReg: r-score =  0.06906265014799318 , MSE =  270.2670437282094 , MAPE =  0.3174741959022944
#Decision Tree: r-score =  0.2943551034715307 , MSE =  204.8607891142896 , MAPE =  0.2632827766998192
#Random Forest: r-score =  0.33745079689923907 , MSE =  192.3493718186192 , MAPE =  0.2533494851491093
#XGBoost: r-score =  0.35155470318214177 , MSE =  188.25476646552448 , MAPE =  0.253455214202358

RF: r-score =  0.33745079689923907 , MSE =  192.3493718186192 , MAPE =  0.2533494851491093


In [18]:
sns.regplot(y_test, y_pred_rf, scatter_kws={"color": "green"}, line_kws={"color": "blue"}) 

/Users/shantellesmith/opt/anaconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:xlabel='metascore'>

In [ ]:
#xgboost_reg_model.best_ntree_limit

100

In [20]:
y_pred_rf = random_forest_reg_model.predict(X_test)

In [23]:
# save in JSON format
#xgboost_reg_model.save_model("../../models/xgboost_reg_model_metascore.pkl")
joblib.dump(random_forest_reg_model, "random_forest_reg_model_metascore.joblib")

['random_forest_reg_model_metascore.joblib']